In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split


events_num_identifier = "_15000_events"
special_dataLabel = "_only3mom"
subdir = "wSim_wReco"
save_data = True
path = "/nfs/dust/belle2/user/axelheim/MC_studies/my6modes/"
root_path = path + "rootfiles/" + subdir + '/'
file_pi = uproot.open((root_path + "pi_nTuples_mode1_10_events.root"))

#file_raw = uproot.open(root_path + "mode0_17_events.root")

In [2]:
file_pi["variables"].keys()


['__experiment__',
 '__run__',
 '__event__',
 '__candidate__',
 '__ncandidates__',
 '__weight__',
 'M',
 'x',
 'y',
 'z',
 'px',
 'py',
 'pz',
 'pt',
 'p',
 'E',
 'kaonID',
 'pionID',
 'isSignal',
 'mcErrors',
 'mcPDG',
 'mcPhotos',
 'mcPrimary',
 'mcInitial',
 'charge',
 'uniqueParticleIdentifier',
 'genMotherID',
 'genMotherPDG',
 'genMotherPDG__bo0__bc',
 'genMotherPDG__bo1__bc',
 'genMotherPDG__bo2__bc',
 'genMotherPDG__bo3__bc']

In [3]:
def conditions(s):
    label = -1
    if int(s['genPDG0']) == 0:
        label = 0 # background, cause not related to MC Particles
    elif (int(s['genPDG0']) == Bs_pdg) or (int(s['genPDG1']) == Bs_pdg) or (int(s['genPDG2']) == Bs_pdg) or (int(s['genPDG3']) == Bs_pdg):
        label = 2 # Bs
    elif (int(s['genPDG0']) == Hc_pdg) or (int(s['genPDG1']) == Hc_pdg) or (int(s['genPDG2']) == Hc_pdg) or (int(s['genPDG3']) == Hc_pdg):    
        label = 3 # Hc
    else: 
        label = 1 # X
    
    return label

In [4]:
Bs_pdg_list = [-511, 511,-521,-521,521,521]
Hc_pdg_list = [-411, 411, -411, -421, 411, 423]
dfs =[]
for mode in range(6):
    print("mode:",mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode)
    
    Bs_pdg = Bs_pdg_list[mode]
    Hc_pdg = Hc_pdg_list[mode]
    
    nm = f"_nTuples_mode{mode}" + events_num_identifier + ".root"
    print((root_path + "gamma" + nm))
    raw_nTuples = uproot.concatenate([(root_path + "gamma" + nm), (root_path + "pi" + nm), (root_path + "K" + nm)])
    df = pd.DataFrame({"event" : np.array(raw_nTuples["__event__"]),
       "mcPDG" : np.array(raw_nTuples["mcPDG"]),
       "mcPrimary":np.array(raw_nTuples["mcPrimary"]),
       "isSignal":np.array(raw_nTuples["isSignal"]),
       #"mcInitial":np.array(raw_nTuples["mcInitial"]),
       "px" :  np.array(raw_nTuples["px"]),
       "py" : np.array(raw_nTuples["py"]),
       "pz" : np.array(raw_nTuples["pz"]),
       "E" : np.array(raw_nTuples["E"]),

       "pionID":np.array(raw_nTuples["pionID"]),   
       "kaonID": np.array(raw_nTuples["kaonID"]),
       "M" : np.array(raw_nTuples["M"]),
       "charge" : np.array(raw_nTuples["charge"]),
       #"fromY4S" : hasAncestor_300553,
       "genPDG0" : np.array(raw_nTuples["genMotherPDG__bo0__bc"]),
       "genPDG1" : np.array(raw_nTuples["genMotherPDG__bo1__bc"]),
       "genPDG2" : np.array(raw_nTuples["genMotherPDG__bo2__bc"]),
       "genPDG3" :  np.array(raw_nTuples["genMotherPDG__bo3__bc"]) })
    df['label'] = df.apply(conditions, axis=1)
    

    
    #print(df)
    
    dfs.append(df)
    print('\n')
    
    # labels: bg=0 , X=1 , Bs=2 , Hc=3
    label_cut_name_list = [["BsHc",1],["XHc",2],["BsX",3],["BsXHc",-10]]
    for label_cut_names in label_cut_name_list:
        label_cut = label_cut_names[1]
        label_cut_name = label_cut_names[0]
        print(label_cut,label_cut_name,label_cut,label_cut_name,label_cut,label_cut_name)
        #print(df.describe())

        df_cut = df.copy()
        if label_cut_name != "BsXHc":
            df_cut = df_cut[df_cut.label != label_cut]
            
            #important to fix labels (no gap in label classes allowed for NN)
            # only first two cases need relabeling, for third BsX deletion is sufficient
            if label_cut<3: 
                mask = df_cut.label == 3
                column_name = 'label'
                df_cut.loc[mask, column_name] = label_cut
                #df_cut[df_cut.label == 2] = label_cut

       # print(df_cut.describe())
        
        # read out data to save it
        #event_tmp = df_cut.copy()        
        #event_tmp = event_tmp[event_tmp.event == 1]
        #print(event_tmp.describe())

        numFSPs_df = pd.DataFrame({'count' : df_cut.groupby( [ "event"] ).size()}).reset_index()
        minFSPs = numFSPs_df["count"].min()
        maxFSPs = numFSPs_df["count"].max()
        print("minFSPs:",minFSPs)
        print("maxFSPs:",maxFSPs,'\n')
        
        df_cut['numFSPs'] = df_cut.groupby('event')['event'].transform('count')
        
        
        #TODO!!
        for num_FSPs_toData in range(minFSPs, maxFSPs+1):
            #print("num_FSPs_toData:",num_FSPs_toData)

            df_num_subset = df_cut.copy()
            df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
        

            numEvents = df_num_subset.event.nunique()
            num_features = 3 
            print("numEvents:",numEvents)
            print("num_FSPs_toData:",num_FSPs_toData)  
            if numEvents == 0:
                print("skipped because empty \n")
                continue
            
            if numEvents < 10:
                print("skipped because <10 events \n")
                continue

            
            leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
            SA_target =  np.zeros((numEvents, num_FSPs_toData))
            global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
            
            event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["event"].unique()
            #print("len(event_list):",len(event_list))
            for i in range(numEvents):
                
                event_iter = event_list[i]
                
                global_tag_masterInfo = str(mode) + "_evt" + str(event_iter)
                global_tag[i,-1] = global_tag_masterInfo
                #print("global_tag[i,-1]:",global_tag[i,-1])
                #print("i:",i,"event_iter:",event_iter)
                #event_df = df_cut.copy()
                #event_df = event_df[event_df.event == event_iter]
                event_df = df_num_subset[df_num_subset.event == event_iter]

                for j in range(num_FSPs_toData):
                    #print("numParticle:",j)
                    particle = event_df.iloc[j]

                    #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                    leaves[i,j,0] = particle["px"]
                    leaves[i,j,1] = particle["py"]
                    leaves[i,j,2] = particle["pz"]
                    #leaves[i,j,3] = particle["E"]

                    global_tag[i,j] = particle["mcPDG"]
                    #global_tag[i,j] += "_"
                    global_tag[i,j] += (particle["mcPrimary"])
                    #global_tag[i,j] += "_"
                    global_tag[i,j] += (particle["isSignal"])
                    SA_target[i,j] = int(particle["label"])
            
                del event_df

            for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
                perms = np.random.permutation(leaves.shape[1])

                leaves[idx,:] = leaves[idx,perms]
                SA_target[idx,:] = SA_target[idx,perms]
                global_tag[idx,0:-1] = global_tag[idx,perms]

            data_subdir = subdir + "_" + label_cut_name + events_num_identifier + special_dataLabel + "/"        
            data_dir = Path("/nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/" + data_subdir)    
            data_dir.mkdir(parents=True, exist_ok=True)


            train_ratio = 0.75
            validation_ratio = 0.15
            test_ratio = 0.10
            
            #print("leaves.shape:",leaves.shape)
            #print("SA_target.shape:",leaves.shape)
            #print("global_tag.shape:",leaves.shape)
            
            x=leaves
            y=SA_target
            z=global_tag

            x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
            x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

            if save_data==True:
                np.save(data_dir / "leaves_train_{}_FSP{}.npy".format(mode,num_FSPs_toData), x_train)
                np.save(data_dir / "is_left_arr_train_{}_FSP{}.npy".format(mode,num_FSPs_toData), y_train)
                np.save(data_dir / "global_tag_train_{}_FSP{}.npy".format(mode,num_FSPs_toData), z_train)

                np.save(data_dir / "leaves_val_{}_FSP{}.npy".format(mode,num_FSPs_toData), x_val)
                np.save(data_dir / "is_left_arr_val_{}_FSP{}.npy".format(mode,num_FSPs_toData), y_val)
                np.save(data_dir / "global_tag_val_{}_FSP{}.npy".format(mode,num_FSPs_toData), z_val)

                np.save(data_dir / "leaves_test_{}_FSP{}.npy".format(mode,num_FSPs_toData), x_test)
                np.save(data_dir / "is_left_arr_test_{}_FSP{}.npy".format(mode,num_FSPs_toData), y_test)
                np.save(data_dir / "global_tag_test_{}_FSP{}.npy".format(mode,num_FSPs_toData), z_test)
            
            print("Data saved to:", data_dir,'is', save_data ,'\n')
            print("")
            del df_num_subset

        
        del df_cut
        
        

mode: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
/nfs/dust/belle2/user/axelheim/MC_studies/my6modes/rootfiles/wSim_wReco/gamma_nTuples_mode0_15000_events.root


1 BsHc 1 BsHc 1 BsHc
minFSPs: 1
maxFSPs: 19 

numEvents: 1
num_FSPs_toData: 1
skipped because <10 events 

numEvents: 29
num_FSPs_toData: 2
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 159
num_FSPs_toData: 3
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 431
num_FSPs_toData: 4
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 1027
num_FSPs_toData: 5
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 1887
num_FSPs_toData: 6
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_

Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 2189
num_FSPs_toData: 9
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 2512
num_FSPs_toData: 10
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 2357
num_FSPs_toData: 11
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 1889
num_FSPs_toData: 12
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 1311
num_FSPs_toData: 13
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 764
num_FSPs_toData: 14
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6

Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 231
num_FSPs_toData: 11
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 85
num_FSPs_toData: 12
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 27
num_FSPs_toData: 13
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 14
num_FSPs_toData: 14
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 2
num_FSPs_toData: 15
skipped because <10 events 

-10 BsXHc -10 BsXHc -10 BsXHc
minFSPs: 1
maxFSPs: 20 

numEvents: 5
num_FSPs_toData: 1
skipped because <10 events 

numEvents: 38
num_FSPs_toData: 2
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/

Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 90
num_FSPs_toData: 2
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 336
num_FSPs_toData: 3
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 950
num_FSPs_toData: 4
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 1807
num_FSPs_toData: 5
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 2525
num_FSPs_toData: 6
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsX_15000_events_only3mom is True 


numEvents: 2826
num_FSPs_toData: 7
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wRec

Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_XHc_15000_events_only3mom is True 


numEvents: 1527
num_FSPs_toData: 3
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_XHc_15000_events_only3mom is True 


numEvents: 2758
num_FSPs_toData: 4
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_XHc_15000_events_only3mom is True 


numEvents: 3402
num_FSPs_toData: 5
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_XHc_15000_events_only3mom is True 


numEvents: 2843
num_FSPs_toData: 6
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_XHc_15000_events_only3mom is True 


numEvents: 1931
num_FSPs_toData: 7
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_XHc_15000_events_only3mom is True 


numEvents: 1021
num_FSPs_toData: 8
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_

Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 2231
num_FSPs_toData: 9
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 1520
num_FSPs_toData: 10
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 920
num_FSPs_toData: 11
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 414
num_FSPs_toData: 12
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 165
num_FSPs_toData: 13
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 64
num_FSPs_toData: 14
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data



1 BsHc 1 BsHc 1 BsHc
minFSPs: 1
maxFSPs: 18 

numEvents: 4
num_FSPs_toData: 1
skipped because <10 events 

numEvents: 34
num_FSPs_toData: 2
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 169
num_FSPs_toData: 3
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 566
num_FSPs_toData: 4
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 1275
num_FSPs_toData: 5
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 2021
num_FSPs_toData: 6
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsHc_15000_events_only3mom is True 


numEvents: 2739
num_FSPs_toData: 7
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsH

Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 1457
num_FSPs_toData: 11
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 888
num_FSPs_toData: 12
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 501
num_FSPs_toData: 13
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 211
num_FSPs_toData: 14
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 86
num_FSPs_toData: 15
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/wSim_wReco_BsXHc_15000_events_only3mom is True 


numEvents: 44
num_FSPs_toData: 16
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes

In [5]:
Bs_pdg = 511
Hc_pdg = 411
nm = f"_nTuples_mode1" + events_num_identifier + ".root"
raw_nTuples = uproot.concatenate([(root_path + "gamma" + nm), (root_path + "pi" + nm), (root_path + "K" + nm)])
event = np.array(raw_nTuples["__event__"])
mcPDG = np.array(raw_nTuples["mcPDG"])
px = np.array(raw_nTuples["px"])
py = np.array(raw_nTuples["py"])
pz = np.array(raw_nTuples["pz"]) #mcPrimary
mcInitial = np.array(raw_nTuples["mcInitial"])
mcPrimary = np.array(raw_nTuples["mcPrimary"])
E = np.array(raw_nTuples["E"])
M = np.array(raw_nTuples["M"])
isSignal = np.array(raw_nTuples["isSignal"])
charge = np.array(raw_nTuples["charge"])
kaonID = np.array(raw_nTuples["kaonID"])
pionID = np.array(raw_nTuples["pionID"])
#hasAncestor_300553 = np.array(raw_nTuples["hasAncestor(300553)"])
genMotherPDG0 = np.array(raw_nTuples["genMotherPDG__bo0__bc"])
genMotherPDG1 = np.array(raw_nTuples["genMotherPDG__bo1__bc"])
genMotherPDG2 = np.array(raw_nTuples["genMotherPDG__bo2__bc"])
genMotherPDG3 = np.array(raw_nTuples["genMotherPDG__bo3__bc"]) 

#u,c = np.unique(event,return_counts=True)
#print('\n entries per event:',u,c)


df = pd.DataFrame({"event" : event,
               "mcPDG" : mcPDG,
               "mcPrimary":mcPrimary,
               "isSignal":isSignal,
               #"mcInitial":mcInitial,
               "px" : px,
               "py" : py,
               "pz" : pz,
               "E" : E,

               "pionID":pionID,   
               "kaonID":kaonID,
               "M" : M,
               "charge" : charge,
               #"fromY4S" : hasAncestor_300553,
               "genPDG0" : genMotherPDG0,
               "genPDG1" : genMotherPDG1,
               "genPDG2" : genMotherPDG2,
               "genPDG3" : genMotherPDG3})
df['label'] = df.apply(conditions, axis=1)

In [6]:
dfs[3][dfs[3].event==1].sort_values("label")

,event,mcPDG,mcPrimary,isSignal,px,py,pz,E,pionID,kaonID,M,charge,genPDG0,genPDG1,genPDG2,genPDG3,label
6,1,NaN,NaN,NaN,-0.074836,0.018443,-0.124483,0.146413,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0
2,1,22.0,1.0,1.0,-0.361160,-0.350682,-0.061892,0.507193,NaN,NaN,0.000000,0.0,111.0,213.0,521.0,300553.0,1
4,1,22.0,1.0,1.0,-0.201496,-0.094562,-0.074745,0.234797,NaN,NaN,0.000000,0.0,111.0,213.0,521.0,300553.0,1
0,1,22.0,0.0,1.0,0.093109,0.145431,0.106453,0.202859,NaN,NaN,0.000000,0.0,-211.0,-521.0,300553.0,0.0,2
3,1,22.0,0.0,1.0,0.004253,0.112888,-0.028027,0.116392,NaN,NaN,0.000000,0.0,-211.0,-521.0,300553.0,0.0,2
5,1,211.0,1.0,0.0,0.031815,0.129437,-0.081710,0.156341,NaN,NaN,0.000000,0.0,-521.0,300553.0,0.0,0.0,2
68791,1,-211.0,1.0,1.0,2.088494,-0.430028,1.018039,2.366984,9.798776e-01,0.009366,0.139570,-1.0,-521.0,300553.0,0.0,0.0,2
120392,1,-321.0,1.0,1.0,-1.044177,0.340423,-0.388148,1.265136,1.398664e-16,1.000000,0.493677,-1.0,421.0,423.0,-521.0,300553.0,2
1,1,22.0,0.0,1.0,0.368030,0.151758,0.145102,0.423711,NaN,NaN,0.000000,0.0,111.0,321.0,-421.0,521.0,3


In [7]:
bin_n = 20
bins = np.linspace(0, bin_n, bin_n)

new[(new.mcPDG==22.0) & (new.mcPrimary==1)]["count"].hist(bins=bins, label='gammas')
new[(new.mcPDG==211.0) & (new.mcPrimary==1)]["count"].hist(bins=bins, label='pions')
new[(new.mcPDG==321.0) & (new.mcPrimary==1)]["count"].hist(bins=bins, legend=True)


NameError: name 'new' is not defined

In [ ]:
numFSPs_df = pd.DataFrame({'count' : df.groupby( [ "event"] ).size()}).reset_index()
minFSPs = numFSPs_df["count"].min()
maxFSPs = numFSPs_df["count"].max()


print("maxFSPs:",maxFSPs)
print("minFSPs:",minFSPs)

bins = np.linspace(0, maxFSPs, maxFSPs)
numFSPs_df["count"].hist(bins=bins, legend=True)


In [ ]:
for i in range(3):
    evNum = i+1
    print("event",evNum)
    print("# FSPs:",len(df[df.event == evNum]))
    print("# pions:",len(df[(df.event == evNum) & (abs(df.mcPDG) == 211)]))
    print("# kaons:",len(df[(df.event == evNum) & (abs(df.mcPDG) == 321)]))
    print("# gammas:",len(df[(df.event == evNum) & (abs(df.mcPDG) == 22)]))
    print('\n')
    
    

In [ ]:
print(len(df[df.event == 1]))
df[df.event == 1]